In [1]:
'''

資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內

'''
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install line-bot-sdk flask flask-ngrok

     |████████████████████████████████| 87 kB 2.7 MB/s 
     |████████████████████████████████| 1.1 MB 24.1 MB/s 
     |████████████████████████████████| 144 kB 50.6 MB/s 
     |████████████████████████████████| 94 kB 2.6 MB/s 
     |████████████████████████████████| 271 kB 68.0 MB/s 


In [3]:
!pip install pyngrok
!pyngrok authtoken 29JlwqUGLZxPpC0QfilJEO8AwsW_xxCAFaGtukqQ8krP2CFd

     |████████████████████████████████| 745 kB 4.0 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=6abe5336dcdbb8cc23e253278b67b0936643aef36362ee36b70060894b3c783f
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [4]:
'''
引用套件
'''

# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)


In [5]:
'''
建置主程序app

建置handler與 line_bot_api
'''

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件
line_bot_api = LineBotApi("7EViQGhjLoXD6HnX+H2llWpD5+bO3sEXJZ/E85+X6YCwNud3ukbdZryinFuVzq0yoLlycjYLx4aqu/DwLhgiuCuzmWfN9Rqt93/4DbuaELQoORWV4OSAA/POyr7vkL+Ur96YcMEguV51FyKT5oVfoAdB04t89/1O/w1cDnyilFU=")
handler = WebhookHandler("4c7bc83333634370cd348be01acb374d")

In [6]:
'''
建置主程序的API入口
  接受Line傳過來的消息
  並取出消息內容
  將消息內容存在google drive的檔案內
  並請handler 進行消息驗證與轉發
'''


# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [10]:
'''

若收到圖片消息時，

先回覆用戶文字消息，並從Line上將照片拿回。

'''

from linebot.models import(
    MessageEvent,ImageMessage, TextSendMessage
)

@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='Image has Upload'+ ' ' + event.message.id))
    message_content = line_bot_api.get_message_content(event.message.id)
    with open(event.message.id+'.jpg', 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)

In [7]:
'''
影片消息，留給你
'''

from linebot.models import(
    MessageEvent,ImageMessage, TextSendMessage, VideoMessage
)

@handler.add(MessageEvent, message=VideoMessage)
def handle_video_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='Video has Upload'+ ' ' + event.message.id))
    message_content = line_bot_api.get_message_content(event.message.id)
    with open(event.message.id+'.mp4', 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)

In [8]:
'''
音訊消息，留給你
'''
from linebot.models import(
    MessageEvent,ImageMessage, TextSendMessage, VideoMessage, AudioMessage
)

@handler.add(MessageEvent, message=AudioMessage)
def handle_audio_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='Audio has Upload'+ ' ' + event.message.id))
    message_content = line_bot_api.get_message_content(event.message.id)
    with open(event.message.id+'.mp3', 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)

In [11]:
# 主程序運行
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e450-35-229-37-228.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [18/May/2022 07:28:37] "POST / HTTP/1.1" 200 -


{"destination":"Ufc94cda1b809d6632a36da9035938144","events":[]}
{"destination":"Ufc94cda1b809d6632a36da9035938144","events":[{"type":"message","message":{"type":"image","id":"16106922272660","contentProvider":{"type":"line"}},"webhookEventId":"01G3B139JMP7PR63MFTYQA9J0K","deliveryContext":{"isRedelivery":false},"timestamp":1652858922322,"source":{"type":"user","userId":"Ua27054d19893e9f37f8babc1e76b6e2f"},"replyToken":"9de05c6cfaa54e0c812234c617076fa6","mode":"active"}]}


127.0.0.1 - - [18/May/2022 07:28:45] "POST / HTTP/1.1" 200 -
